In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import datetime

## Reading in the Data

In [2]:
entries = pd.read_csv("entries.csv")
buses = pd.read_csv("buses.csv")

stop_loop_mapping = pd.read_csv("stop_loop_mapping.csv")
loops = pd.read_csv("stops.csv")

users = pd.read_csv("users.csv")

list_of_df = [entries, buses, stop_loop_mapping, loops, users]

## Checking out the columns for each

In [3]:
for i in list_of_df:
    print(i.columns)

Index(['boarded', 'stop_id', 'timestamp', 'date_added', 'loop_id', 'driver_id',
       'id', 'left_behind', 'bus_id', 'is_deleted'],
      dtype='object')
Index(['id', 'bus_number', 'is_deleted'], dtype='object')
Index(['id', 'loop_id', 'stop_id', 'display_order', 'is_deleted'], dtype='object')
Index(['id', 'stop_name', 'is_deleted'], dtype='object')
Index(['id', 'first_name', 'last_name', 'password', 'email', 'is_deleted'], dtype='object')


## How to Find the busiest bus for the whole year

* I would solve this by aggregating then summing the data

* Then, you can use the pd.datetimeindex() function in in combination with datetime.py to find which day of the week, month, and semester each date represents.

* Then, then we can aggregate by each level of granularity

In [4]:
aggregated = entries.groupby('bus_id').sum('boarded')
aggregated = aggregated.sort_values(by = ['boarded'], ascending = False)
aggregated.reset_index(inplace = True)
aggregated['bus_id'] = aggregated['bus_id'].astype(object)



The busiest bus, which I am denoting as the bus that had the most people board throughout the year, is below:

In [5]:
aggregated.loc[:, ['bus_id', 'boarded']]

,bus_id,boarded
0,8,74717
1,9,68793
2,7,67589
3,5,55224
4,15,52827
5,16,50383
6,13,45716
7,17,45146
8,6,40137
9,14,33125


Bus 8 was the busiest for the whole year.

Now, finding the busiest bus for the other granularities of dates.

## Busiest bus by Day of Week

In [6]:
entries['date_added'] = pd.to_datetime(entries['date_added'])

entries['day'] = pd.DatetimeIndex(entries['date_added']).dayofweek
entries['month'] = pd.DatetimeIndex(entries['date_added']).month
entries['quarter'] = pd.DatetimeIndex(entries['date_added']).quarter

In [7]:
by_day = entries.groupby(['day', 'bus_id']).sum('boarded')

by_day = by_day.sort_values(by = ['boarded'], ascending = False)
by_day.reset_index(inplace = True)


by_day.loc[:, ['day', 'bus_id', 'boarded']].head()



,day,bus_id,boarded
0,1,7,27562
1,1,8,15480
2,2,8,15385
3,3,8,15057
4,0,9,14382


The busiest day of the week for the busiest bus is Tuesday. If we are just looking at the busiest day of the week overall, it is listed below.

In [8]:
busy_day = (by_day.groupby('day').sum('boarded').sort_values(by = ['boarded'], ascending = False))


In [9]:
busy_day.loc[:, 'boarded']

day
1    119920
2    106165
3    104044
0    102906
4     84654
6     21296
5       163
Name: boarded, dtype: int64

Tuesday is the busiest day overall as well.

## Busiest bus Month

#### Takeaways so far:

* Bus 8 was the busiest overall

* Bus 7 was the busiest on the busiest day of the week (Tuesday)



In [10]:
by_month = entries.groupby(['month', 'bus_id']).sum('boarded')

by_month = by_month.sort_values(by = ['boarded'], ascending = False)
by_month.reset_index(inplace = True)


by_month.loc[:, ['month', 'bus_id', 'boarded']].head()

,month,bus_id,boarded
0,2,7,24872
1,9,8,16004
2,9,9,14232
3,11,8,13419
4,10,8,12655


The busiest month for the busiest bus is February. If we are just looking at the busiest month overall, it is listed below.

In [11]:
busy_month = (by_month.groupby('month').sum('boarded').sort_values(by = ['boarded'], ascending = False))

In [12]:
busy_month.loc[:, 'boarded']

month
9     94435
11    82392
10    80672
2     68417
3     55667
4     49731
12    45234
8     30197
1     25199
5      7204
Name: boarded, dtype: int64

September was the busiest month overall.

## Busiest bus by Semester

For this, we need to assign some conditions so that we can create a new column titled 'semester' based on the values of the 'quarter' column.

In [13]:
conditions = [
    (entries['quarter'] == 1) | (entries['quarter'] == 2),
    (entries['quarter'] == 3) | (entries['quarter'] == 4)
]

values = ['Spring', 'Fall']

In [14]:
entries['semester'] = np.select(conditions, values)

In [15]:
entries.head()

,boarded,stop_id,timestamp,date_added,loop_id,driver_id,id,left_behind,bus_id,is_deleted,day,month,quarter,semester
0,0,75,1/16/2021 17:00,2021-01-16,9,50,804685,0,8,0,5,1,1,Spring
1,0,99,1/16/2021 17:04,2021-01-16,15,49,804686,0,17,0,5,1,1,Spring
2,1,97,1/16/2021 17:06,2021-01-16,15,49,804687,0,17,0,5,1,1,Spring
3,0,81,1/16/2021 17:07,2021-01-16,15,49,804688,0,17,0,5,1,1,Spring
4,0,79,1/16/2021 17:08,2021-01-16,15,49,804689,0,17,0,5,1,1,Spring


In [16]:
by_semester = entries.groupby(['semester', 'bus_id']).sum('boarded')

by_semester = by_semester.sort_values(by = ['boarded'], ascending = False)
by_semester.reset_index(inplace = True)


by_semester.loc[:, ['semester', 'bus_id', 'boarded']].head()

,semester,bus_id,boarded
0,Fall,8,53796
1,Spring,7,45948
2,Fall,9,44699
3,Fall,5,42224
4,Fall,15,36873


It looks like the busiest semester for the busiest bus was the Fall semester. Bus 8 wins here. If we are only interested in the semester in which busses carried the most people, the results are listed below.

In [17]:
busy_semester = (by_semester.groupby('semester').sum('boarded').sort_values(by = ['boarded'], ascending = False))

In [18]:
busy_semester.loc[:, 'boarded']

semester
Fall      332930
Spring    206218
Name: boarded, dtype: int64

The Fall semester was the busiest.

## By Bus Summary

In [19]:
by_bus_summary = pd.DataFrame(dict({"Granularity": ["Year", "Semester", "Month", "Day of Week"],
                      "Bus_id": [aggregated['bus_id'][0], by_semester['bus_id'][0],
                                by_month['bus_id'][0], by_day['bus_id'][0]]}))
by_bus_summary

,Granularity,Bus_id
0,Year,8
1,Semester,8
2,Month,7
3,Day of Week,7


## Finding the busiest stops

* Similar to above, I will aggregate the data first.

* However, I might include how many people were left behind in the calculation.

* I will include an overall stop retention rate to be more thorough.



In [20]:
temp1 = entries.groupby('stop_id').sum()
temp1['stop_retention'] = temp1['left_behind']/(temp1['left_behind'] + temp1['boarded'])

In [21]:
temp1 = temp1.sort_values(by = 'stop_retention', ascending = False)
temp1.reset_index(inplace = True)

temp1.loc[:, ['stop_id', 'stop_retention']].head()

,stop_id,stop_retention
0,118,0.014442
1,72,0.007614
2,80,0.007004
3,77,0.003884
4,92,0.003831


Stop 118 had the greatest retention rate overall for the entire year.

## Finding the Busiest Stop by Day of Week

* To do this, I will do what I did earlier for the bus_id's

* I will group by the identifier as well as the granularity of the date, then produce the table

In [22]:
temp2 = entries.groupby(['day', 'stop_id']).sum()

In [23]:
temp2['stop_retention'] = temp2['left_behind']/(temp2['left_behind'] + temp2['boarded'])

temp2 = temp2.sort_values(by = 'stop_retention', ascending = False)
temp2.reset_index(inplace = True)

temp2.loc[:, ['stop_id', 'day', 'stop_retention']].head()

,stop_id,day,stop_retention
0,72,4,0.022809
1,118,3,0.021129
2,118,0,0.019534
3,118,2,0.014138
4,118,1,0.014088


On Fridays, stop 72 had the greatest retention rate.

## Finding the Busiest Stop by Month


In [24]:
temp3 = entries.groupby(['month', 'stop_id']).sum()

temp3['stop_retention'] = temp3['left_behind']/(temp3['left_behind'] + temp3['boarded'])

temp3 = temp3.sort_values(by = 'stop_retention', ascending = False)
temp3.reset_index(inplace = True)

temp3.loc[:, ['stop_id', 'month', 'stop_retention']].head()


,stop_id,month,stop_retention
0,72,9,0.030802
1,118,8,0.021030
2,118,3,0.020501
3,118,11,0.018577
4,118,9,0.018449


In September, stop 72 had the greatest retention rate.

## Finding the Busiest Stop by Semester

In [25]:
temp4 = entries.groupby(['semester', 'stop_id']).sum()

temp4['stop_retention'] = temp4['left_behind']/(temp4['left_behind'] + temp4['boarded'])

temp4 = temp4.sort_values(by = 'stop_retention', ascending = False)
temp4.reset_index(inplace = True)

temp4.loc[:, ['stop_id', 'semester', 'stop_retention']].head()


,stop_id,semester,stop_retention
0,118,Fall,0.016165
1,72,Fall,0.014108
2,80,Fall,0.008280
3,77,Fall,0.005604
4,80,Spring,0.004831




In the Fall semester, stop 118 had the greatest retention rate.

## Finding the Busiest Hours

* This one will be the most difficult, mainly due to coming up with a meaningful way to display the results

* For the sake of the analysis, I will keep the hour listed for each timestamp as the hour of day, i.e, I won't round up to the next hour

* For this, we will need to create a new column with the hour of the day. This should be straightforward

* The ordinal day of the year will also be needed



In [26]:
entries['hour'] = pd.DatetimeIndex(entries['timestamp']).hour
entries['ordinal_day'] = pd.DatetimeIndex(entries['date_added']).dayofyear

In [27]:
entries.head()

,boarded,stop_id,timestamp,date_added,loop_id,driver_id,id,left_behind,bus_id,is_deleted,day,month,quarter,semester,hour,ordinal_day
0,0,75,1/16/2021 17:00,2021-01-16,9,50,804685,0,8,0,5,1,1,Spring,17,16
1,0,99,1/16/2021 17:04,2021-01-16,15,49,804686,0,17,0,5,1,1,Spring,17,16
2,1,97,1/16/2021 17:06,2021-01-16,15,49,804687,0,17,0,5,1,1,Spring,17,16
3,0,81,1/16/2021 17:07,2021-01-16,15,49,804688,0,17,0,5,1,1,Spring,17,16
4,0,79,1/16/2021 17:08,2021-01-16,15,49,804689,0,17,0,5,1,1,Spring,17,16


In [28]:
## Checking if the number of days is correct

entries['date_added'].max()

Timestamp('2021-12-17 00:00:00')

In [29]:
entries['date_added'].min()

Timestamp('2021-01-16 00:00:00')

In [30]:
len(entries['date_added'].unique())

196

It looks like the busses do not operate every day of the year, which makes sense.

In [31]:
busy_hours = entries.groupby(['hour', 'ordinal_day']).sum('boarded')

busy_hours.sort_values(by = ['boarded'], ascending = False)
busy_hours.reset_index(inplace = True)



In [32]:
busy_hours['hour'] = busy_hours['hour'].astype(object)

In [35]:
fig = px.line(busy_hours, x = 'ordinal_day', y = 'boarded', color = 'hour', 
             color_discrete_map = {
                 "6": ""
             })

fig.show()

With this kind of figure, you can select specific periods throughout the year, as well as deselect any hour line you would like. 

Some takeaways from this figure:

* The spike on ordinal day 40 occurred on February which was one of the days Indiana got a lot of snowfall in 2021

* The period of no people riding the bus initially confused me, but then I remembered that the busses might not operate during the summer.